In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import importlib.metadata
import seaborn as sns
import time
import scipy.stats as stats

from keras.src.layers import Bidirectional, LSTM
from sklearn import metrics
from sklearn import model_selection

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


## Load and pre-process the data set

In [2]:
train = pd.read_csv('Data/train.csv')
# train = pd.read_csv('Data/preprocessed_train.csv')
print('loaded %d records' % len(train))

# Make sure all comment_text values are strings
train['comment_text'] = train['comment_text'].astype(str) 

# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

train = convert_dataframe_to_bool(train)

loaded 1804874 records


## Split the data into 80% train and 20% validate sets

In [ ]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.2)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))


## Create a text tokenizer

In [13]:
MAX_NUM_WORDS = 10000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

# Create a text tokenizer.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])

# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 250
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

## Define and train a Convolutional Neural Net for classifying toxic comments

In [16]:
from tqdm import tqdm
from tqdm.keras import TqdmCallback

EMBEDDINGS_PATH = 'Embedding_file/glove.6B.100d.txt'
EMBEDDINGS_DIMENSION = 100
DROPOUT_RATE = 0.3
LEARNING_RATE = 0.00005
NUM_EPOCHS = 10
BATCH_SIZE = 128

from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import RMSprop

print("Loading BERT model...")
bert_model = TFBertModel.from_pretrained('Models/bert-base-uncased')
print("BERT model loaded.")
print("Loading tokenizer...")
tokenizer = BertTokenizer.from_pretrained('Models/bert-base-uncased')
print("Tokenizer loaded.")

def encode_texts(texts):
    encoded_texts = []
    for text in tqdm(texts.tolist(), desc="Encoding texts"):
        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        encoded_texts.append(encoded_text)
    return encoded_texts

with tf.device('/GPU:0'):
    # Prepare data
    train_encodings = encode_texts(train_df[TEXT_COLUMN])
    train_labels = to_categorical(train_df[TOXICITY_COLUMN])
    validate_encodings = encode_texts(validate_df[TEXT_COLUMN])
    validate_labels = to_categorical(validate_df[TOXICITY_COLUMN])


Loading BERT model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

BERT model loaded.
Loading tokenizer...
Tokenizer loaded.


Encoding texts: 100%|██████████| 360975/360975 [05:07<00:00, 1173.34it/s]


ValueError: Exception encountered when calling layer 'tf_bert_model_1' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_1' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 250), dtype=int32, sparse=None, name=keras_tensor_14>
  • attention_mask=<KerasTensor shape=(None, 250), dtype=int32, sparse=None, name=keras_tensor_15>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [17]:
def train_model():
    # Create model layers.
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='sequence_input')
    attention_mask_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='attention_mask_input')

    # BERT Layers
    bert_output = bert_model(sequence_input, attention_mask=attention_mask_input).pooler_output

    # Dropout Layers
    x = Dropout(DROPOUT_RATE)(bert_output)

    # Dense Layers
    x = Dense(128, activation='relu')(x)

    # Predict Layers
    preds = Dense(2, activation='softmax')(x)

    # Compile model.
    model = Model(inputs=[sequence_input, attention_mask_input], outputs=preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=LEARNING_RATE),
                  metrics=['acc'])

    # Train model with TQDMCallback.
    tqdm_callback = TqdmCallback(verbose=2)  # verbose=2 for less verbosity

    model.fit([train_encodings['input_ids'], train_encodings['attention_mask']],
              train_labels,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=([validate_encodings['input_ids'], validate_encodings['attention_mask']], validate_labels),
              verbose=0,  # Set verbose to 0 to not duplicate progress bar
              callbacks=[tqdm_callback])  # Add tqdm_callback to callbacks

    return model


In [18]:
with tf.device('/GPU:0'):
    model = train_model()

ValueError: Exception encountered when calling layer 'tf_bert_model_1' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_1' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 250), dtype=int32, sparse=None, name=sequence_input>
  • attention_mask=<KerasTensor shape=(None, 250), dtype=int32, sparse=None, name=attention_mask_input>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

## Generate model predictions on the validation set

In [6]:
MODEL_NAME = 'Bert_model'
# save the model
model.save('Models/Bert.h5')
validate_df[MODEL_NAME] = model.predict(pad_text(validate_df[TEXT_COLUMN], tokenizer))[:, 1]

22561/22561 ━━━━━━━━━━━━━━━━━━━━ 118s 5ms/step


In [7]:
validate_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,my_model
2400071,NaN,False,question wisdom ready change system soon good ...,NaN,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000124
660555,1050181.0,False,want weiner investigation go discover informat...,0.0,0.0,0.0,0.000000,0.0,False,False,...,approved,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.003940
554174,921208.0,False,killer moroccan origin report show publish ja...,0.0,0.1,0.3,0.000000,0.3,False,False,...,approved,0.0,0.0,0.0,0.0,0.0,0.0,6.0,10.0,0.099911
410934,745991.0,False,shameless bravado juvenile posturing lefty exp...,0.0,0.0,0.0,0.166667,0.0,False,False,...,rejected,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.093263
489957,844575.0,False,repub plan plan donald j. trump new presiden...,0.0,0.0,0.0,0.000000,0.0,False,False,...,approved,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.001096


## Define bias metrics, then evaluate our new model for bias using the validation set predictions

In [8]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = pd.concat([subgroup_negative_examples, non_subgroup_positive_examples])
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = pd.concat([subgroup_positive_examples, non_subgroup_negative_examples])
    return compute_auc(examples[label], examples[model_name])


def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {'subgroup': subgroup, 'subgroup_size': len(dataset[dataset[subgroup]]),
                  SUBGROUP_AUC: compute_subgroup_auc(dataset, subgroup, label_col, model),
                  BPSN_AUC: compute_bpsn_auc(dataset, subgroup, label_col, model),
                  BNSP_AUC: compute_bnsp_auc(dataset, subgroup, label_col, model)}
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df


,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,2191,0.779656,0.751227,0.953948
6,black,2969,0.779857,0.712884,0.964288
7,white,4964,0.786009,0.720060,0.965577
5,muslim,4253,0.801811,0.776812,0.954333
4,jewish,1541,0.842341,0.852811,0.939993
8,psychiatric_or_mental_illness,975,0.842646,0.835229,0.948722
1,female,10700,0.867986,0.852470,0.947230
0,male,8891,0.870325,0.842911,0.953353
3,christian,8042,0.887223,0.908032,0.925454


## Calculate the final score

In [9]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME))

0.8750458072206464

## Prediction on Test data

In [21]:
test = pd.read_csv('Data/test.csv')
submission = pd.read_csv('Data/sample_submission.csv', index_col='id')

In [23]:
submission['prediction'] = model.predict(pad_text(test[TEXT_COLUMN], tokenizer))[:, 1]
submission.to_csv('benchmark_submission.csv')

3042/3042 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step
